#Creade data from csv files

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

violation_fact = pd.read_csv('FactViolation')
violation_dim = pd.read_csv('Violation')
visit_fact = pd.read_csv('FactVisit')
restaurant_dim = pd.read_csv('Restaurant')
grade_fact = pd.read_csv('FactGrade')


In [2]:
visit_fact['ViolationCount'] = visit_fact[visit_fact['VisitHadViolation'] == True].groupby('ViolationID')['VisitHadViolation'].transform('count')
visit_fact['ViolationCount'] = visit_fact['ViolationCount'].fillna(0).astype(int)
visit_unique = visit_fact[['ViolationID','ViolationCount']].drop_duplicates(subset=['ViolationID','ViolationCount'])
violation_detail = pd.merge(visit_unique,violation_dim,on='ViolationID',how='inner').drop(['ViolationCode','ViolationID'],axis=1)

violation_detail.to_csv('ViolationTypeCount',index=False)
violation_detail.sort_values('ViolationCount',ascending=False).iloc[:20]


,ViolationCount,ViolationDescription,IsViolationCritical,ViolationMainType
0,53836,Non-food contact surface improperly constructe...,Not Critical,Food Surface
12,42043,Facility not vermin proof. Harborage or condit...,Not Critical,Vermin
34,28179,Evidence of mice or live mice present in facil...,Critical,Rodents
2,25898,"Food contact surface not properly washed, rins...",Critical,Food Surface
28,22690,Food not protected from potential source of co...,Critical,Contamination
6,21955,Plumbing not properly installed or maintained;...,Not Critical,Plumbing
22,21669,Cold food item held above 41º F (smoked fish a...,Critical,Temperature
4,20083,Filth flies or food/refuse/sewage-associated (...,Critical,Flies
10,19327,Hot food item not held at or above 140º F.,Critical,Temperature
24,8413,Live roaches present in facility's food and/or...,Critical,Roaches


#Violation by restaurant

In [3]:
violation_status = pd.merge(visit_fact,violation_dim,on='ViolationID',how='left').drop(['FactVisitID','ViolationCode','InspectionTypeID'],axis=1)

violation_critical = violation_status[(violation_status['IsViolationCritical'] == 'Critical') & (violation_status['VisitHadViolation'] == True)].copy()
violation_critical['RestaurantCriticalViolationCount'] = violation_critical.groupby('RestaurantID')['VisitHadViolation'].transform('count')
violation_unique = violation_critical[['RestaurantID','RestaurantCriticalViolationCount']].drop_duplicates(subset=['RestaurantID','RestaurantCriticalViolationCount'])
violation_name = pd.merge(violation_unique,restaurant_dim,on='RestaurantID',how='inner').drop(['BORO','Street','Building','Zipcode','Cuisine','RestaurantID'],axis=1)

violation_notcritical = violation_status[(violation_status['IsViolationCritical'] == 'Not Critical') & (violation_status['VisitHadViolation'] == True)].copy()
violation_notcritical['RestaurantNoneCriticalViolationCount'] = violation_notcritical.groupby('RestaurantID')['VisitHadViolation'].transform('count')
violation_unique_none = violation_notcritical[['RestaurantID','RestaurantNoneCriticalViolationCount']].drop_duplicates(subset=['RestaurantID','RestaurantNoneCriticalViolationCount'])
violation_name_none = pd.merge(violation_unique_none,restaurant_dim,on='RestaurantID',how='inner').drop(['BORO','Street','Building','Zipcode','Cuisine','RestaurantID'],axis=1)

violation_union = pd.merge(violation_name,violation_name_none,on='Name',how='inner')

violation_union.to_csv('RestaurantViolationCount',index=False)
violation_union.sort_values('RestaurantCriticalViolationCount',ascending=False).iloc[:20]

,RestaurantCriticalViolationCount,Name,RestaurantNoneCriticalViolationCount
0,2072,DUNKIN,2166
389,1383,SUBWAY,1420
235,922,MCDONALD'S,1029
562,734,STARBUCKS,1314
150,627,KENNEDY FRIED CHICKEN,577
98,586,CROWN FRIED CHICKEN,492
194,452,BURGER KING,534
61,444,POPEYES,487
277,419,GOLDEN KRUST CARIBBEAN BAKERY & GRILL,451
586,388,"DUNKIN',' BASKIN ROBBINS",379


In [4]:
grade_fact['Inspection'] = grade_fact['InspectionDate'].astype(str) + grade_fact['RestaurantID'].astype(str)
grade_inspection = grade_fact[['Inspection', 'Grade']].drop_duplicates(subset='Inspection')
grade_inspection.groupby(grade_fact['Inspection'].str[:4])['Grade'].value_counts()


Inspection  Grade
2016        A         2979
            B          240
            C           67
            P           42
2017        A        13547
            B         1148
            C          368
            P          361
            Z            1
2018        A        20592
            B         2107
            C          752
            P          611
            N           15
            Z            4
            G            1
2019        A        24539
            B         2492
            C          856
            P          572
            Z          163
            N          120
            G            1
2020        A         5335
            B          477
            N          419
            C          127
            Z           92
            P           19
2021        A          648
            N          205
            Z           10
Name: Grade, dtype: int64

#Score

In [16]:
score_inspection = grade_fact[['Inspection', 'Score']].drop_duplicates(subset='Inspection')
score_inspection.groupby(grade_fact['InspectionDate'].astype(str).str[:4])['Score'].mean()

InspectionDate
2016    13.900114
2017    14.820798
2018    15.246219
2019    15.850593
2020    15.635797
2021    18.161213
Name: Score, dtype: float64

In [5]:
violation_critical['CriticalViolationRestaurantCount'] = violation_critical.groupby(['ViolationMainType','RestaurantID'])['VisitHadViolation'].transform('count')

restaurant_unique = violation_critical[['ViolationMainType','RestaurantID','CriticalViolationRestaurantCount']].drop_duplicates(subset=['RestaurantID','CriticalViolationRestaurantCount'])
restaurant_name = pd.merge(restaurant_unique,restaurant_dim,on='RestaurantID',how='inner').drop(['BORO','Street','Building','Zipcode','Cuisine','RestaurantID'],axis=1)

restaurant_name.head(20)
restaurant_name.to_csv('CriticalViolationRestaurantCount',index=False)
restaurant_name.sort_values('CriticalViolationRestaurantCount',ascending=False).iloc[:20]


,ViolationMainType,CriticalViolationRestaurantCount,Name
1,Contamination,529,DUNKIN
0,Food Surface,469,DUNKIN
1450,Food Surface,301,SUBWAY
2,Flies,272,DUNKIN
1447,Temperature,259,SUBWAY
1445,Rodents,248,SUBWAY
2092,Flies,220,STARBUCKS
868,Flies,213,MCDONALD'S
4,Temperature,209,DUNKIN
867,Food Surface,180,MCDONALD'S
